In [ ]:
# Install TensorFlow and all required libraries
!pip install tensorflow numpy matplotlib scikit-learn seaborn
# ============================================
# MNIST Digit Classification
# Simple Feedforward Neural Network
# Framework: TensorFlow/Keras
# ============================================

# Section 1: Import Libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

print("TensorFlow version:", keras.__version__)

# ============================================
# Section 2: Load and Explore Data
# ============================================

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print("Training data shape:", x_train.shape)  # (60000, 28, 28)
print("Training labels shape:", y_train.shape)  # (60000,)
print("Test data shape:", x_test.shape)  # (10000, 28, 28)
print("Test labels shape:", y_test.shape)  # (10000,)

# Visualize some examples
plt.figure(figsize=(10, 4))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(x_train[i], cmap='gray')
    plt.title(f"Label: {y_train[i]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

# ============================================
# Section 3: Data Preprocessing
# ============================================

print("\n--- Data Preprocessing ---")

# Step 1: Flatten images from 28x28 to 784
x_train_flat = x_train.reshape(-1, 784)  # -1 means "figure out this dimension"
x_test_flat = x_test.reshape(-1, 784)

print("Flattened training shape:", x_train_flat.shape)  # (60000, 784)
print("Flattened test shape:", x_test_flat.shape)  # (10000, 784)

# Step 2: Normalize pixel values from 0-255 to 0-1
x_train_normalized = x_train_flat.astype('float32') / 255.0
x_test_normalized = x_test_flat.astype('float32') / 255.0

print("Original pixel range:", x_train_flat[0].min(), "-", x_train_flat[0].max())
print("Normalized pixel range:", x_train_normalized[0].min(), "-", x_train_normalized[0].max())

# Step 3: One-hot encode labels
# Before: y_train[0] = 5
# After: y_train_encoded[0] = [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
y_train_encoded = to_categorical(y_train, 10)
y_test_encoded = to_categorical(y_test, 10)

print("\nOriginal label:", y_train[0])
print("One-hot encoded label:", y_train_encoded[0])

# ============================================
# Section 4: Build Model Architecture
# ============================================

print("\n--- Building Neural Network ---")

model = keras.Sequential([
    # Input layer (implicit - defined by first hidden layer input shape)
    
    # Hidden Layer 1: 128 neurons with ReLU activation
    layers.Dense(128, activation='relu', input_shape=(784,), name='hidden_layer_1'),
    
    # Hidden Layer 2: 64 neurons with ReLU activation
    layers.Dense(64, activation='relu', name='hidden_layer_2'),
    
    # Output Layer: 10 neurons with Softmax activation
    layers.Dense(10, activation='softmax', name='output_layer')
])

# Display model architecture
model.summary()

# ============================================
# Section 5: Compile Model
# ============================================

print("\n--- Compiling Model ---")

model.compile(
    optimizer='adam',  # Adam optimizer - smart learning rate adjustment
    loss='categorical_crossentropy',  # Loss function for multi-class classification
    metrics=['accuracy']  # Track accuracy during training
)

print("Model compiled successfully!")

# ============================================
# Section 6: Train Model
# ============================================

print("\n--- Training Model ---")

# Train for 10 epochs with 20% validation split
history = model.fit(
    x_train_normalized,
    y_train_encoded,
    epochs=10,
    batch_size=128,
    validation_split=0.2,  # Use 20% of training data for validation
    verbose=1
)

# ============================================
# Section 7: Visualize Training Progress
# ============================================

# Plot training history
plt.figure(figsize=(12, 4))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# ============================================
# Section 8: Evaluate Model on Test Set
# ============================================

print("\n--- Evaluating Model on Test Set ---")

test_loss, test_accuracy = model.evaluate(
    x_test_normalized,
    y_test_encoded,
    verbose=0
)

print(f"\nFinal Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Final Test Loss: {test_loss:.4f}")

# ============================================
# Section 9: Test on Random Images
# ============================================

print("\n--- Testing on Random Images ---")

# Select 5 random images from test set
num_samples = 5
random_indices = np.random.randint(0, len(x_test), num_samples)

plt.figure(figsize=(12, 4))

for i, idx in enumerate(random_indices):
    # Get image and true label
    image = x_test[idx]
    true_label = y_test[idx]
    
    # Prepare image for prediction
    image_flat = image.reshape(1, 784).astype('float32') / 255.0
    
    # Make prediction
    prediction = model.predict(image_flat, verbose=0)
    predicted_label = np.argmax(prediction)
    confidence = np.max(prediction) * 100
    
    # Display
    plt.subplot(1, num_samples, i+1)
    plt.imshow(image, cmap='gray')
    plt.title(f"True: {true_label}\nPred: {predicted_label}\n({confidence:.1f}%)")
    plt.axis('off')
    
    # Print detailed info
    print(f"\nImage {i+1}:")
    print(f"  True Label: {true_label}")
    print(f"  Predicted Label: {predicted_label}")
    print(f"  Confidence: {confidence:.2f}%")
    print(f"  Correct: {'✓' if predicted_label == true_label else '✗'}")

plt.tight_layout()
plt.show()

# ============================================
# Section 10: Confusion Matrix (Bonus)
# ============================================

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Get predictions for all test images
y_pred = model.predict(x_test_normalized, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)

# Create confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Print classification report
print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred_classes))
